In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
# Convert categorical columns to numeric using one-hot encoding
dataset = pd.get_dummies(dataset, columns=['sex', 'smoker', 'region'], drop_first=True)

In [ ]:
from sklearn.model_selection import train_test_split

# Separate features and target variable
X = dataset.drop('expenses', axis=1)
y = dataset['expenses']

# Split the data
train_dataset, test_dataset, train_labels, test_labels = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [ ]:
# Normalize the features
normalizer = tf.keras.layers.Normalization()
normalizer.adapt(np.array(train_dataset))


In [ ]:
#Building the model
model = keras.Sequential([
    normalizer,
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    loss='mean_absolute_error',
    metrics=['mae', 'mse']
)


In [ ]:
# Train the model
history = model.fit(
    train_dataset,
    train_labels,
    validation_split=0.2,
    verbose=1,
    epochs=100
)


In [ ]:
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
